# Import Library and Dataset

In [ ]:
import pandas as pd
import gdown
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error

In [ ]:
# Helper Function
def import_from_gdrive(id_file, file_name, sheet_name=None):
    link = 'https://drive.google.com/uc?id=' + id_file
    output_file = '/content/' + file_name
    gdown.download(link, output_file, quiet=False)

    df = pd.read_excel(output_file)
    return df

In [ ]:
id_file = '1iNMFgJmFTkSWK9VnMf_uhlKYYmNclJNm'
df = import_from_gdrive(id_file,'final_dataset_converted_rupiah.xlsx')

Downloading...
From: https://drive.google.com/uc?id=1iNMFgJmFTkSWK9VnMf_uhlKYYmNclJNm
To: /content/final_dataset_converted_rupiah.xlsx
100%|██████████| 15.1M/15.1M [00:00<00:00, 31.1MB/s]


In [ ]:
# df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34008 entries, 0 to 34007
Data columns (total 68 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Total Household Income                               34008 non-null  float64
 1   Region                                               34008 non-null  object 
 2   Total Food Expenditure                               34008 non-null  float64
 3   Main Source of Income                                34008 non-null  object 
 4   Agricultural Household indicator                     34008 non-null  int64  
 5   Bread and Cereals Expenditure                        34008 non-null  float64
 6   Total Rice Expenditure                               34008 non-null  float64
 7   Meat Expenditure                                     34008 non-null  float64
 8   Total Fish and  marine products Expenditure          34008 non-nul

In [ ]:
fix_columns =['Total Household Income',
              'Household Head Sex',
              'Household Head Age',
              'Household Head Marital Status',
              # 'Household Head Highest Grade Completed',
              'Household Head Highest Grade Completed (Simplified)',
              'Total Household Income',
              'Region',
              'Type of Building/House',
              'House Floor Area',
              'Number of bedrooms',
              'Electricity',
              'Tenure Status',
              'Type of Household',
              'Total Number of Family members',
              'Total number of family members employed',
              # 'Number of Car, Jeep, Van',
              # 'Number of Motorcycle/Tricycle',
              # 'Number of Television',
              # 'Number of Airconditioner',
              # 'Number of Personal Computer',
              # 'Number of Cellular phone',
              # 'Number of Refrigerator/Freezer',
              # 'Number of Washing Machine',
              'Number of Kids',
              'Number of Vehicles',
              'Number of Communication Devices',
              'Number of Electronics',
              ]

X =  df.loc[:, fix_columns]
X = X.loc[:,~X.columns.duplicated()]

# Mengubah tipe data int menjadi float
int_columns = X.select_dtypes(include=['int32', 'int64']).columns
X[int_columns] = X[int_columns].astype('float64')

y = df['Total Expenses']

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34008 entries, 0 to 34007
Data columns (total 18 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Total Household Income                               34008 non-null  float64
 1   Household Head Sex                                   34008 non-null  object 
 2   Household Head Age                                   34008 non-null  float64
 3   Household Head Marital Status                        34008 non-null  object 
 4   Household Head Highest Grade Completed (Simplified)  33999 non-null  object 
 5   Region                                               34008 non-null  object 
 6   Type of Building/House                               34008 non-null  object 
 7   House Floor Area                                     34008 non-null  float64
 8   Number of bedrooms                                   34008 non-nul

In [ ]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Pipeline preprocessing untuk fitur numerik
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())  # Standarisasi
])
# Pipeline preprocessing untuk fitur kategorikal
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())  # One-hot encoding
])

# Gabungkan pipeline preprocessing untuk fitur numerik dan kategorikal
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X = preprocessor.fit_transform(X)

In [ ]:
TEST_SIZE = 0.2
RANDOM_STATE = 42
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
n_cols = X_train.shape[1]

print("X_train shape : " , X_train.shape)
print("y_train shape : " , y_train.shape)
print("X_test  shape : " , X_test.shape)
print("y_test  shape : " , y_test.shape)

X_train shape :  (27206, 59)
y_train shape :  (27206,)
X_test  shape :  (6802, 59)
y_test  shape :  (6802,)


In [ ]:
# Bangun model regresi linear dengan regularisasi L2
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.01))
])

# Kompilasi model
model.compile(optimizer='adam', loss='mse')

# Latih model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=2)

# Evaluasi model
loss = model.evaluate(X_test, y_test, verbose=0)
print('Mean Squared Error on Test Set:', loss)

# Prediksi
predictions = model.predict(X_test)

Epoch 1/100
851/851 - 3s - loss: 17508932398350336.0000 - 3s/epoch - 4ms/step
Epoch 2/100
851/851 - 3s - loss: 17474640305717248.0000 - 3s/epoch - 3ms/step
Epoch 3/100
851/851 - 2s - loss: 17372946217566208.0000 - 2s/epoch - 2ms/step
Epoch 4/100
851/851 - 2s - loss: 17186693887033344.0000 - 2s/epoch - 2ms/step
Epoch 5/100
851/851 - 1s - loss: 16908547509977088.0000 - 1s/epoch - 1ms/step
Epoch 6/100
851/851 - 1s - loss: 16534964812120064.0000 - 1s/epoch - 1ms/step
Epoch 7/100
851/851 - 1s - loss: 16062195213271040.0000 - 1s/epoch - 2ms/step
Epoch 8/100
851/851 - 1s - loss: 15487896882511872.0000 - 1s/epoch - 1ms/step
Epoch 9/100
851/851 - 1s - loss: 14817167946022912.0000 - 1s/epoch - 1ms/step
Epoch 10/100
851/851 - 1s - loss: 14054316256002048.0000 - 1s/epoch - 2ms/step
Epoch 11/100
851/851 - 1s - loss: 13203269560041472.0000 - 1s/epoch - 2ms/step
Epoch 12/100
851/851 - 2s - loss: 12275238796525568.0000 - 2s/epoch - 2ms/step
Epoch 13/100
851/851 - 2s - loss: 11286933535719424.0000 - 2s